In [3]:
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
np.random.seed(0)
import math # to help with data reshaping of the data

import numpy as np
import torch
torch.manual_seed(0)

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter


import tqdm
import matplotlib.pyplot as plt
import logging

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


import sys
sys.path.append('../')
sys.path.append('../../src')

from pose_gru import PoseGRU_inputFC2
from benji_prox_dataloader import *

import wandb
# _ = wandb.init(project="vh-human-motion-pred", entity="benjidayan", name="GRU_joints_21_05")


numpy: 1.22.3


In [4]:
root_dir = "/cluster/scratch/bdayan/prox_data/PROXD/"
smplx_model_path='/cluster/home/bdayan/prox/prox/models_smplx_v1_1/models/'

batch_size = 15
in_frames=10
pred_frames=5
lr=0.0001
n_iter = 10
save_every=20

save_folder = 'saves'
os.makedirs(save_folder, exist_ok=True)

save_path=os.path.join(save_folder, 'PoseGRU_inputFC2_21_05_epoch{epoch}_bn{batchnum}.pt')
save_path.format(epoch=3, batchnum=5)

pd = proxDatasetSkeleton(root_dir=root_dir, in_frames=in_frames, pred_frames=pred_frames, output_type='joint_locations', smplx_model_path=smplx_model_path)

dataloader = DataLoader(pd, batch_size=batch_size,
                        shuffle=True, num_workers=0, collate_fn=my_collate)

'saves/PoseGRU_inputFC2_21_05_epoch3_bn5.pt'

In [6]:
from utils import *
pd.output_type='raw_pkls'
(idx, (in_frames_fns, in_data), (pred_frames_fns, pred_data)) = pd.__getitem__(500)
_betas, _body_pose, base_euler_angles, base_transl = extract_data(in_data[-1])

In [7]:
base_euler_angles

tensor([[3.7251, 0.1746, 1.2804]])

In [ ]:
criterion = nn.MSELoss()
losses = []
losses_rep = []

# rnn = MockupModel(input_size=21*3, hidden_size=50, num_layers=2, output_size=(pred_frames, 21, 3), seq_len=in_frames)
gru = PoseGRU_inputFC2(input_size=(25,3))

optimizer = torch.optim.Adam(gru.parameters(), lr=lr)


save_path='model_epoch{epoch}_bn{batchnum}.pt'
writer = SummaryWriter()

wandb.config = {
    "learning_rate": lr,
    "epochs": n_iter,
    "batch_size": batch_size,
    "in_frames": in_frames,
    "pred_frames": pred_frames
}

idx_counter = 0
for epoch in range(n_iter):
    for i, (indices, in_skels, fut_skels) in (pbar := tqdm.tqdm(enumerate(dataloader))):
        optimizer.zero_grad()
        
        pred_frames = fut_skels.shape[1]
        
        cur_state, pred_skels = gru.forward_prediction(in_skels, pred_len=pred_frames)
        loss = criterion(pred_skels, fut_skels)
        loss.backward()
        optimizer.step() 

        rep_pred = in_skels[:, -1, :, :]
        a = rep_pred.detach().numpy()
        
        a = np.tile(a, (fut_skels.shape[1], 1, 1, 1))
        rep_pred = torch.Tensor(a).transpose(0, 1)

        loss_rep = criterion(rep_pred, fut_skels)
        losses_rep.append(loss_rep.item())
        
        losses.append(loss.item())
        
        wandb.log({'MSEloss': loss, 'rep_pred_MSEloss': loss_rep})
        
        pbar.set_description(f"avg last 20 loss: {np.mean(losses[-20:]):.4f} avg last 200-100: {np.mean(losses[-200:-100]):.4f}")

        writer.add_scalar('Loss', losses[-1], idx_counter)
        writer.add_scalar('Loss_rep', losses_rep[-1], idx_counter)
        if i % save_every == (save_every-1):
            torch.save({
            'epoch': epoch,
            'batch_num': i,
            'model_state_dict': gru.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, save_path.format(epoch=epoch, batchnum=i))
            
        idx_counter += 1
    print(f'end epoch {epoch}: total mean loss: {np.mean(losses)}')

plt.plot(losses)
print(losses[-4:])